<a href="https://colab.research.google.com/github/Guimarret/Guimarret/blob/main/Workshop_Gemma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2024 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

This is a quick demo of Gemma running on KerasNLP.

Note that you will need a large GPU (e.g. A100) to run this as well.

General Keras reading:
- [Getting started with Keras](https://keras.io/getting_started/)
- [Getting started with KerasNLP](https://keras.io/guides/keras_nlp/getting_started/)

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/gemma-cookbook/blob/main/Gemma/Keras_Gemma_2_Quickstart_Chat.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

# Workshop Gemma - Primeiros Passos com Gemma2 (Chat)

## Access

In [1]:
import os
from google.colab import userdata

os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')

os.environ["KERAS_BACKEND"] = "jax"  # Or "tensorflow" or "torch".

## Installation

In [2]:
# Install all deps
!pip install -U keras-nlp
!pip install -U keras>3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.4/548.4 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 103.0 MB/s eta 0:00:00


## Quickstart

In [3]:
import keras_nlp
import keras

# Run at half precision.
keras.config.set_floatx("bfloat16")

# using 9B instruction-tuned model
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_instruct_2b_en")
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,614,341,888 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,614,341,888 (4.87 GB)

 Trainable params: 2,614,341,888 (4.87 GB)

 Non-trainable params: 0 (0.00 B)

### Define formatting helper functions

In [4]:
from IPython.display import Markdown
import textwrap

def display_chat(prompt, text):
  formatted_prompt = "<font size='+1' color='brown'>🙋‍♂️<blockquote>" + prompt + "</blockquote></font>"
  text = text.replace('•', '  *')
  text = textwrap.indent(text, '> ', predicate=lambda _: True)
  formatted_text = "<font size='+1' color='teal'>🤖\n\n" + text + "\n</font>"
  return Markdown(formatted_prompt+formatted_text)

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

### Create a chat helper to manage the conversation state

In [5]:
class ChatState():
  """
  Manages the conversation history for a turn-based chatbot
  Follows the turn-based conversation guidelines for the Gemma family of models
  documented at https://ai.google.dev/gemma/docs/formatting
  """

  __START_TURN_USER__ = "<start_of_turn>user\n"
  __START_TURN_MODEL__ = "<start_of_turn>model\n"
  __END_TURN__ = "<end_of_turn>\n"

  def __init__(self, model, system=""):
    """
    Initializes the chat state.

    Args:
        model: The language model to use for generating responses.
        system: (Optional) System instructions or bot description.
    """
    self.model = model
    self.system = system
    self.history = []

  def add_to_history_as_user(self, message):
      """
      Adds a user message to the history with start/end turn markers.
      """
      self.history.append(self.__START_TURN_USER__ + message + self.__END_TURN__)

  def add_to_history_as_model(self, message):
      """
      Adds a model response to the history with the start turn marker.
      Model will generate end turn marker.
      """
      self.history.append(self.__START_TURN_MODEL__ + message)

  def get_history(self):
      """
      Returns the entire chat history as a single string.
      """
      return "".join([*self.history])

  def get_full_prompt(self):
    """
    Builds the prompt for the language model, including history and system description.
    """
    prompt = self.get_history() + self.__START_TURN_MODEL__
    if len(self.system)>0:
      prompt = self.system + "\n" + prompt
    return prompt

  def send_message(self, message):
    """
    Handles sending a user message and getting a model response.

    Args:
        message: The user's message.

    Returns:
        The model's response.
    """
    self.add_to_history_as_user(message)
    prompt = self.get_full_prompt()
    response = self.model.generate(prompt, max_length=4096)
    result = response.replace(prompt, "")  # Extract only the new response
    self.add_to_history_as_model(result)
    return result


### Chat with the model

In [6]:
chat = ChatState(gemma_lm)
message = "help me understand how to make coffee"
display_chat(message, chat.send_message(message))

<font size='+1' color='brown'>🙋‍♂️<blockquote>help me understand how to make coffee</blockquote></font><font size='+1' color='teal'>🤖

> Let's brew up some coffee knowledge! ☕ 
> 
> Here's a breakdown of how to make coffee, covering the basics and some popular methods:
> 
> **1. The Essentials:**
> 
> * **Coffee Beans:** Choose your favorite roast (light, medium, dark) and grind them fresh for the best flavor. 
> * **Water:** Use filtered water for the purest taste.
> * **Brewing Method:** This determines how you extract the coffee's flavor.
> * **Equipment:** You'll need a coffee grinder, a coffee maker (drip, French press, pour-over, etc.), a kettle, and a mug.
> 
> **2. Choosing Your Method:**
> 
> * **Drip Coffee Maker:**  The most common and convenient method. Simply add water and coffee grounds, and the machine does the rest.
> * **French Press:**  A simple and elegant method. Steep coarsely ground coffee in hot water for a few minutes, then press the plunger to separate the grounds.
> * **Pour-Over:**  A precise and flavorful method.  Pour hot water over coffee grounds in a filter cone, allowing the water to drip through.
> * **AeroPress:**  A versatile and portable method.  Presses hot water through coffee grounds using a plunger.
> * **Cold Brew:**  Steep coarsely ground coffee in cold water for 12-24 hours for a smooth, less acidic brew.
> 
> **3. Brewing Steps (General):**
> 
> 1. **Grind:** Grind your beans to the appropriate coarseness for your chosen method.
> 2. **Measure:** Use a coffee-to-water ratio of 1:15 (e.g., 1 gram of coffee for every 15 ml of water).
> 3. **Heat Water:**  Bring water to a boil, then let it cool slightly for optimal brewing temperature (195-205°F/90-96°C).
> 4. **Bloom:**  For pour-over and some other methods, pour a small amount of hot water over the grounds to "bloom" them, releasing gases and enhancing flavor.
> 5. **Brew:**  Follow the specific instructions for your chosen method.
> 6. **Serve:**  Enjoy your freshly brewed coffee!
> 
> **4. Tips for Success:**
> 
> * **Freshly Ground Beans:**  The fresher the beans, the better the flavor.
> * **Water Quality:**  Filtered water is best.
> * **Water Temperature:**  Brewing at the right temperature is crucial for optimal extraction.
> * **Grind Size:**  Adjust the grind size based on your brewing method.
> * **Experiment:**  Don't be afraid to experiment with different beans, roasts, and brewing methods to find your perfect cup.
> 
> **5. Resources:**
> 
> * **Coffee Bean Roasters:**  Many roasters offer guides and recipes for different brewing methods.
> * **Online Resources:**  Websites like Sprudge, Serious Eats, and The Kitchn have excellent articles on coffee brewing.
> 
> **Let me know if you'd like a more detailed guide on a specific brewing method!** 
> <end_of_turn>
</font>

In [7]:
message = "what is the best coffee?"
display_chat(message, chat.send_message(message))

<font size='+1' color='brown'>🙋‍♂️<blockquote>what is the best coffee?</blockquote></font><font size='+1' color='teal'>🤖

> Ah, the age-old question! There's no single "best" coffee because taste is subjective and depends on individual preferences. 
> 
> However, we can talk about what makes a coffee great and some popular choices:
> 
> **Factors that make a coffee great:**
> 
> * **Origin:**  Different regions have unique growing conditions, resulting in distinct flavor profiles. 
>     * **Latin America:**  Known for bright acidity, chocolate notes, and fruity flavors (e.g., Colombia, Brazil).
>     * **Africa:**  Often bold, with earthy, spicy, and floral notes (e.g., Ethiopia, Kenya).
>     * **Asia:**  Can be smooth, with hints of spice and floral notes (e.g., Indonesia, Vietnam).
> * **Roast Level:**  
>     * **Light:**  Bright acidity, floral, and fruity.
>     * **Medium:**  Balanced, with a hint of sweetness and acidity.
>     * **Dark:**  Bold, smoky, and often with chocolate or caramel notes.
> * **Brewing Method:**  Different methods extract different flavors. 
>     * **Pour-over:**  Highlights nuanced flavors.
>     * **French Press:**  Full-bodied and rich.
>     * **Espresso:**  Intense and concentrated.
> * **Personal Preference:**  Ultimately, it comes down to what you enjoy! Do you prefer a bright and fruity coffee or a rich and chocolatey one?
> 
> **Popular Coffee Choices:**
> 
> * **Ethiopian Yirgacheffe:**  Known for its floral, citrusy, and tea-like notes.
> * **Kenyan AA:**  Bold, with black currant and berry flavors.
> * **Sumatran Mandheling:**  Earthy, spicy, and full-bodied.
> * **Colombian Supremo:**  Balanced, with notes of chocolate and caramel.
> * **Guatemalan Antigua:**  Smooth, with hints of chocolate and nuts.
> 
> **How to find your "best" coffee:**
> 
> 1. **Explore:**  Try different roasts, origins, and brewing methods.
> 2. **Read Reviews:**  See what other coffee enthusiasts have to say.
> 3. **Visit a Specialty Coffee Shop:**  Sample different beans and ask for recommendations.
> 4. **Experiment:**  Don't be afraid to try new things!
> 
> 
> Remember, the best coffee is the one you enjoy the most! ☕️ 
> <end_of_turn>
</font>

In [8]:
message = "what are similar drinks?"
display_chat(message, chat.send_message(message))

<font size='+1' color='brown'>🙋‍♂️<blockquote>what are similar drinks?</blockquote></font><font size='+1' color='teal'>🤖

> You're looking for drinks similar to coffee, which is a great starting point for exploring the world of beverages!  Here are some options, categorized by what aspect of coffee they share:
> 
> **Similar Flavor Profiles:**
> 
> * **Tea:**  Black tea, green tea, oolong tea, and even herbal infusions can share similar caffeine levels and earthy, sometimes floral notes with coffee.
> * **Chicory Coffee:**  Made from roasted chicory root, it has a similar bitter, earthy flavor to coffee but is naturally caffeine-free.
> * **Matcha:**  A finely ground green tea powder, matcha offers a unique, slightly sweet, and vegetal flavor profile.
> * **Yerba Mate:**  A South American beverage made from the leaves of the yerba mate plant, it has a strong, earthy, and slightly bitter taste.
> 
> **Similar Caffeine Content:**
> 
> * **Energy Drinks:**  These often contain high levels of caffeine, similar to coffee, but with added sugar and other ingredients.
> * **Black Tea:**  Black tea has a moderate caffeine content, similar to coffee.
> * **Green Tea:**  Green tea has a lower caffeine content than coffee, but still provides a stimulating effect.
> 
> **Similar Rituals:**
> 
> * **Hot Chocolate:**  Both are enjoyed warm, often with a touch of sweetness, and can be a comforting ritual.
> * **Chai Tea:**  A spiced tea with black tea, milk, and spices, it offers a warm and comforting experience.
> * **Hot Toddy:**  A warm drink made with whiskey, honey, and lemon, it's a comforting and soothing beverage.
> 
> **Similar Cultural Significance:**
> 
> * **Turkish Coffee:**  A strong, unfiltered coffee brewed in a special pot, it's a cultural staple in Turkey and other parts of the Middle East.
> * **Espresso:**  A concentrated coffee drink from Italy, it's a staple in many cafes and is often enjoyed with pastries or sweets.
> 
> 
> **Beyond the Usual:**
> 
> * **Golden Milk:**  A turmeric-based drink with warming spices, it's a popular alternative for those seeking a healthy and flavorful beverage.
> * **Kombucha:**  A fermented tea drink with a slightly tart and fizzy flavor, it's a refreshing and probiotic-rich option.
> * **Fruit Infused Water:**  A simple and refreshing way to add flavor to your water, it can be customized with your favorite fruits.
> 
> 
> Let me know if you'd like to explore any of these options in more detail! 
> <end_of_turn>
</font>

Use the `get_history` method to see how all the context was retained by the Chat class.

In [9]:
print(chat.get_history())

<start_of_turn>user
help me understand how to make coffee<end_of_turn>
<start_of_turn>model
Let's brew up some coffee knowledge! ☕ 

Here's a breakdown of how to make coffee, covering the basics and some popular methods:

**1. The Essentials:**

* **Coffee Beans:** Choose your favorite roast (light, medium, dark) and grind them fresh for the best flavor. 
* **Water:** Use filtered water for the purest taste.
* **Brewing Method:** This determines how you extract the coffee's flavor.
* **Equipment:** You'll need a coffee grinder, a coffee maker (drip, French press, pour-over, etc.), a kettle, and a mug.

**2. Choosing Your Method:**

* **Drip Coffee Maker:**  The most common and convenient method. Simply add water and coffee grounds, and the machine does the rest.
* **French Press:**  A simple and elegant method. Steep coarsely ground coffee in hot water for a few minutes, then press the plunger to separate the grounds.
* **Pour-Over:**  A precise and flavorful method.  Pour hot water ov